In [1]:
%md
### ⚡Treinamento dos principais comandos de SQL e SPARK

UsageError: Line magic function `%md` not found.


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
%md
#### Create Table

In [ ]:
%sql
--criando tabela pessoa
CREATE OR REPLACE TABLE spark_catalog.bronze.tr_pessoa(
  id INT COMMENT 'Chave Primária',
  primeiro_nome VARCHAR(255) COMMENT 'Primeiro nome da Pessoa',
  ultimo_nome VARCHAR(255)  COMMENT 'Ultimo nome da Pessoa',
  data_nascimento DATE COMMENT 'Data de Nascimento'
) 
LOCATION 'dbfs:/FileStore/landing_zone/tr_pessoa'
TBLPROPERTIES ('comment' = 'Esta tabela contém dados de pessoas');

In [ ]:
%sql
describe spark_catalog.bronze.tr_pessoa

In [ ]:
%sql
--criando tabela rendimento anual pessoa
CREATE OR REPLACE TABLE spark_catalog.bronze.tr_pessoa_rendimento(
  id INT COMMENT 'Chave Primária',
  id_pessoa int COMMENT 'Id pessoa',
  rendimento_anual double COMMENT 'rendimento anual',
  ano_rendimento int COMMENT 'ano rendimento'
) 
LOCATION 'dbfs:/FileStore/landing_zone/tr_pessoa_rendimento'
TBLPROPERTIES ('comment' = 'Esta tabela contém dados de rendimento de pessoas');

In [ ]:
%md
#### Visualizando o schema da Table

In [ ]:
%sql
DESCRIBE spark_catalog.bronze.tr_pessoa

In [ ]:
%sql
DESCRIBE spark_catalog.bronze.tr_pessoa_rendimento

In [ ]:
%sql
-- Inserindo dados na tabela de pessoas
INSERT INTO spark_catalog.bronze.tr_pessoa
 (id, primeiro_nome, ultimo_nome, data_nascimento) 
VALUES
(1, 'Anderson', 'Leal', '2003-01-15'),
(2, 'Felix', 'Souza', '1998-01-20'), 
(3, 'Junior', 'Amorim','1994-02-11'),
(4, 'Barbara', 'Souza','2001-03-22'),
(5, 'Jordam', 'Avilar','2001-03-22')

In [ ]:
%sql
-- Inserindo dados na tabela de rendimento de pessoas
INSERT INTO spark_catalog.bronze.tr_pessoa_rendimento
 (id, id_pessoa, rendimento_anual, ano_rendimento) 
VALUES
(1, 1, 710235, 2022),
(2, 1, 140450, 2023), 
(3, 1, 100000, 2024),
(4, 2, 214700, 2022),
(5, 2, 510235, 2023),
(6, 2, 340450, 2024), 
(7, 3, 180000, 2022),
(8, 3, 204000, 2023),
(9, 3, 717235, 2024),
(10, 4, 140490, 2022),
(12, 4, 214000, 2024)

In [ ]:
%md
#### Lendo : Spark or SQL

In [ ]:
%sql
--Ler passando o Location
SELECT *
FROM delta.`dbfs:/FileStore/landing_zone/tr_pessoa/`

In [ ]:
%sql
--Ler usando a tabela
SELECT *
FROM bronze.tr_pessoa

In [ ]:
%python
df = spark.read.format("delta").load("dbfs:/FileStore/landing_zone/tr_pessoa")
df.display()

In [ ]:
%python
spark.table('spark_catalog.bronze.tr_pessoa_rendimento').display()

In [ ]:
%python
maça = spark.sql('select * from spark_catalog.bronze.tr_pessoa_rendimento')
df.display()

In [ ]:
%md
#### Spark or SQL

In [ ]:
%md
##### Criando um dataframe

In [ ]:
df_pessoa = spark.table('spark_catalog.bronze.tr_pessoa')
df_pessoa_rendimento = spark.sql('select * from spark_catalog.bronze.tr_pessoa_rendimento')

In [ ]:
df_pessoa.display()

In [ ]:
df_pessoa_rendimento.display()

In [ ]:
df_pessoa.select('primeiro_nome','ultimo_nome').display()

In [ ]:
df_pessoa.display()

In [ ]:
df_pessoa2 = df_pessoa.select('primeiro_nome','ultimo_nome','data_nascimento')
df_pessoa2.display()

In [ ]:
df_pessoa.display()

In [ ]:
df_pessoa2.display()

In [ ]:
%sql 
select primeiro_nome from bronze.tr_pessoa

In [ ]:
df_pessoa.filter(col('ultimo_nome') == 'Souza').show()

In [ ]:
df_pessoa.filter((col('ultimo_nome') == 'Souza') & (col('primeiro_nome') == 'Barbara')).show()

In [ ]:
df_pessoa.filter(
    (col('ultimo_nome') == 'Souza') & 
    (col('primeiro_nome') == 'Barbara')
    ).show()

In [ ]:
%sql
select * 
from spark_catalog.bronze.tr_pessoa
where ultimo_nome = 'Souza' -- and primeiro_nome = 'Barbara'

In [ ]:
%python
spark.sql('''
          SELECT * 
          FROM spark_catalog.bronze.tr_pessoa
          WHERE ultimo_nome = "Souza" AND primeiro_nome = "Barbara"
           ''').display()

In [ ]:
df_pessoa = df_pessoa.withColumnRenamed('primeiro_nome', 'nome_principal').display()

In [ ]:
df_pessoa.display()

In [ ]:
%sql
select 
  id, 
  primeiro_nome as nome_principal,
  ultimo_nome,
  data_nascimento
from spark_catalog.bronze.tr_pessoa

In [ ]:
df_pessoa\
    .withColumn('nome', regexp_replace(col('primeiro_nome'),'a','A')).display()

In [ ]:
df_pessoa.withColumn('primeiro_nome', regexp_replace(col('primeiro_nome'),'A','POPOPOPO')).display()